In [ ]:
import pandas as pd
import numpy  as np
from math import sin, cos, pi, sqrt
import random
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()

In [ ]:
_n_circles_       = 25
circle_geoms      = []
_min_circle_sep_  = 20
_half_sep_        = _min_circle_sep_/2.0   # Needs to be more than the _radius_inc_test_
_radius_inc_test_ = 2
_radius_start_    = _radius_inc_test_ + 1  # Needs to be more than the _radius_inc_test_ ... less than the _min_circle_sep_
_escape_px_       = 10                     # less than the _min_circle_sep_

def circleOverlaps(cx, cy, r):
    for _geom_ in circle_geoms:
        dx, dy = _geom_[0] - cx, _geom_[1] - cy
        d      = sqrt(dx*dx+dy*dy)
        if d < (r + _geom_[2] + _min_circle_sep_): # at least 10 pixels apart...
            return True
    return False

def findOpening():
    cx, cy, r = random.randint(30, 570), random.randint(30, 370), random.randint(20,40)
    while circleOverlaps(cx,cy,r):
        cx, cy, r = random.randint(30, 570), random.randint(30, 370), random.randint(20,40)
    return cx, cy, r

for i in range(_n_circles_):
    cx, cy, r = findOpening()
    circle_geoms.append((cx,cy,r))

# points to connect
pts = []
c0  = random.randint(0, len(circle_geoms)-1)
c1 =  random.randint(0, len(circle_geoms)-1)
while c1 == c0:
    c1 =  random.randint(0, len(circle_geoms)-1)
a0, a1 = random.random() * 2 * pi, random.random() * 2 * pi
cx, cy, r = circle_geoms[c0]
pts.append((cx+(r+_radius_start_+1)*cos(a0), cy+(r+_radius_start_+1)*sin(a0)))
pts.append((cx+(r+_escape_px_)*cos(a0), cy+(r+_escape_px_)*sin(a0)))
cx, cy, r = circle_geoms[c1]
pts.append((cx+(r+_escape_px_)*cos(a1), cy+(r+_escape_px_)*sin(a1)))
pts.append((cx+(r+_radius_start_+1)*cos(a1), cy+(r+_radius_start_+1)*sin(a1)))

svg = '<svg width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in circle_geoms:
    _color_ = '#000000'
    _dist_, _inter_  = rt.segmentIntersectsCircle((pts[1],pts[2]),_geom_)
    if _dist_ <= _geom_[2]:
        _color_ = '#ff0000'
    svg += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'

_path_ = f'M {pts[0][0]} {pts[0][1]}'
for i in range(1, len(pts)):
    _path_ += f' L {pts[i][0]} {pts[i][1]}'
svg += f'<path d="{_path_}" fill="none" stroke="#ff0000" stroke-width="0.2" />'

svg += f'<circle cx="{pts[0][0]}"  cy="{pts[0][1]}"  r="3" stroke="#000000" fill="#ff0000" />'
svg += f'<circle cx="{pts[-1][0]}" cy="{pts[-1][1]}" r="3" stroke="#000000" fill="#ff0000" />'

svg += '</svg>'
rt.displaySVG(svg)

In [ ]:
def breakSegment(_segment_):
    if rt.segmentLength(_segment_) < 2.0:
        return _segment_
    for _geom_ in circle_geoms:
        _circle_plus_ = (_geom_[0], _geom_[1], _geom_[2]+_radius_inc_test_)
        _dist_, _inter_  = rt.segmentIntersectsCircle(_segment_,_circle_plus_)
        if _dist_ <= _circle_plus_[2]:
            if _inter_[0] == _geom_[0] and _inter_[1] == _geom_[1]:
                dx, dy   = _segment_[1][0] - _segment_[0][0], _segment_[1][1] - _segment_[0][1]
                l        = sqrt(dx*dx+dy*dy)
                dx,  dy  = dx/l, dy/l
                pdx, pdy = -dy, dx 
                return [(_segment_[0][0], _segment_[0][1]), (_geom_[0] + pdx*(_geom_[2]+_half_sep_), _geom_[1] + pdy*(_geom_[2]+_half_sep_)), (_segment_[1][0], _segment_[1][1])]
            else:
                dx, dy = _inter_[0] - _geom_[0], _inter_[1] - _geom_[1]
                l      = sqrt(dx*dx+dy*dy)
                dx, dy = dx/l, dy/l
                return [(_segment_[0][0], _segment_[0][1]), (_geom_[0] + dx*(_geom_[2]+_half_sep_), _geom_[1] + dy*(_geom_[2]+_half_sep_)), (_segment_[1][0], _segment_[1][1])]
    return _segment_
breakSegment([pts[1],pts[2]])

In [ ]:
last_length = 0
_segments_  = []
for _pt_ in pts:
    _segments_.append(_pt_)
iter = 0
while last_length != len(_segments_):
    last_length    = len(_segments_)
    _new_segments_ = []
    for i in range(len(_segments_)-1):
        _new_ = breakSegment([_segments_[i], _segments_[i+1]])
        if len(_new_) == 3:
            _new_segments_.append(_new_[0])
            _new_segments_.append(_new_[1])
        else:
            _new_segments_.append(_new_[0])
    _new_segments_.append(_new_[-1])
    _segments_ = _new_segments_
    iter += 1

svg2 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in circle_geoms:
    _color_ = '#000000'
    svg2 += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'
_path_ = f'M {_segments_[0][0]} {_segments_[0][1]}'
for i in range(1,len(_segments_)):
    _path_ += f' L {_segments_[i][0]} {_segments_[i][1]}'

svg2 += f'<path d="{_path_}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg2 += f'<circle cx="{pts[0][0]}"  cy="{pts[0][1]}"  r="3" stroke="#000000" fill="#ff0000" />'
svg2 += f'<circle cx="{pts[-1][0]}" cy="{pts[-1][1]}" r="3" stroke="#000000" fill="#ff0000" />'
svg2 += '</svg>'

rt.tile([svg,svg2])

In [ ]:
svg3 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in circle_geoms:
    _color_ = '#000000'
    svg3 += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'


_path_ =  f'M {_segments_[0][0]} {_segments_[0][1]}'
_path_ += f' L {_segments_[1][0]} {_segments_[1][1]}'
for i in range(1,len(_segments_)-2):
    v0 = rt.unitVector([_segments_[i],   _segments_[i-1]])
    v1 = rt.unitVector([_segments_[i+1], _segments_[i+2]])
    _amp_ = 10.0
    _path_ += f' C {_segments_[i][0]-_amp_*v0[0]} {_segments_[i][1]-_amp_*v0[1]} {_segments_[i+1][0]-_amp_*v1[0]} {_segments_[i+1][1]-_amp_*v1[1]} {_segments_[i+1][0]} {_segments_[i+1][1]}'
_path_ += f' L {_segments_[-1][0]} {_segments_[-1][1]}'

svg3 += f'<path d="{_path_}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg3 += f'<circle cx="{pts[0][0]}"  cy="{pts[0][1]}"  r="1" stroke="#000000" fill="#ff0000" />'
svg3 += f'<circle cx="{pts[-1][0]}" cy="{pts[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg3 += '</svg>'

rt.tile([svg2,svg3])